# YMBOT-231 - Crops analysis

## Prepare:
1. Imports
2. Aliases
3. Data paths and results

In [1]:
import os
import pandas as pd
from PIL import ImageDraw
from typing import List
import json
from pyzbar.pyzbar import decode as zbar_decode
import cv2

from ymbot_cv_utils.toloka.outputs import RectangleOutput
from ymbot_cv_utils.toloka.shapes import RectangleShape
from ymbot_cv_utils.ya_pic_decoder.barcode_decoder import BarcodeDecoder
from ymbot_cv_utils.utils.image_loader import ImageLoader
from ymbot_cv_utils.ya_pic_decoder.code_info import CodeInfo

In [2]:
# useful aliases for Toloka lebeling file

a_id = "id"
a_input = "image_url"
a_output_result = "output:result"
a_output_confidence = "output:confidence"
a_output_empty = "output:noObjects"

# result columns
a_res_pack_no = "pack_id"
a_res_ya_result = "yandex_decoding_result"
a_res_cv_result = "cv_decoding_result"
a_res_zbar_result = "zbar_decoding_result"
a_res_border_intersection = "intersection_with_border"
a_res_near_border = "near_border"
a_res_crop_filename = "crop_filename"
a_res_visualize_filename = "visualisation_filename"
a_res_crop_laplacian_score = "laplacian_score"

result_columns = [
    a_id,
    a_input,
    a_res_pack_no,
    a_output_empty,
    a_output_confidence,
    a_res_ya_result,
    a_res_cv_result,
    a_res_zbar_result,
    a_res_near_border,
    a_res_border_intersection,
    a_res_crop_filename,
    a_res_crop_laplacian_score,
    a_res_visualize_filename,
]

# parameters
BORDER_SIZE = 50  # px
BORDER_INTERSECTION = 0.25


## Calculating

1. Create empty dataframe
2. Iterate on each type of labeled codes
   1. Open and prepare df with tsv data
   2. For each record:
      - loading image
      - crop
      - decoding
      - write results


In [3]:
# Reading file
tsv_filename = "./data/plt-qr-minidataset.tsv"
# tsv df prepare
df = pd.read_csv(tsv_filename, sep='\t')
df = df.dropna(how='all', axis=0).dropna(how='all', axis=1)
df.reset_index(drop=True)


,system:itemId,id,image_url,output:_404,output:confidence,output:noObjects,output:result
0,R9k0g54g6y2i95pdDAvR,0,https://mr-toloka-images.s3.yandex.net/QR_PLTD...,False,0.991624,False,"[{""top"":0.45725548164572555,""left"":0.630697216..."
1,3KaOb2gb3AaTQrky725J,1,https://mr-toloka-images.s3.yandex.net/QR_PLTD...,False,0.992858,False,"[{""top"":0.46322855468868285,""left"":0.584481369..."
2,y5VawLgwQZdFKa4VxjeD,2,https://mr-toloka-images.s3.yandex.net/QR_PLTD...,False,0.992232,False,"[{""top"":0.46630784111773504,""left"":0.548001292..."
3,76L1JDWJ3AOUlD2aLnz3,3,https://mr-toloka-images.s3.yandex.net/QR_PLTD...,False,0.991751,False,"[{""top"":0.46772567162168593,""left"":0.490635888..."
4,EMep8Px86ybFw04ZnKeb,4,https://mr-toloka-images.s3.yandex.net/QR_PLTD...,False,0.991402,False,"[{""top"":0.46233829041228386,""left"":0.453955587..."
...,...,...,...,...,...,...,...
255,A3Mp8RV86PJU893ANRW2,255,https://mr-toloka-images.s3.yandex.net/QR_PLTD...,False,0.993884,False,"[{""top"":0.15682073137674915,""left"":0.406231726..."
256,R9k0g54g6e8i95pdDAvR,256,https://mr-toloka-images.s3.yandex.net/QR_PLTD...,False,0.993617,False,"[{""top"":0.16481410401277496,""left"":0.404302976..."
257,3KaOb2gb3NjfQrky725J,257,https://mr-toloka-images.s3.yandex.net/QR_PLTD...,False,0.993884,False,"[{""top"":0.17803009806370826,""left"":0.403986141..."
258,y5VawLgwQpliKa4VxjeD,258,https://mr-toloka-images.s3.yandex.net/QR_PLTD...,False,0.993617,False,"[{""top"":0.193615114352068,""left"":0.40364858176..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   system:itemId      260 non-null    object 
 1   id                 260 non-null    int64  
 2   image_url          260 non-null    object 
 3   output:_404        260 non-null    bool   
 4   output:confidence  260 non-null    float64
 5   output:noObjects   260 non-null    bool   
 6   output:result      260 non-null    object 
dtypes: bool(2), float64(1), int64(1), object(3)
memory usage: 10.8+ KB


In [5]:
from dataclasses import dataclass

@dataclass
class CropInfo:
    filename: str
    ya_result: str
    cv2_result: str
    zbar_result: str
    intersection_with_border: float = 0.0
    laplacian_score: float = 0.0

In [6]:
import cv2
import numpy as np
from PIL import Image

def image_to_cv(image:Image.Image):
    open_cv_image = np.array(image) 
    # Convert RGB to BGR 
    open_cv_image = open_cv_image[:, :, ::-1].copy() 
    return open_cv_image

def variance_of_laplacian(image):
	return cv2.Laplacian(image, cv2.CV_64F).var()

In [7]:


def get_rectangle_intersection(shape_a: RectangleShape, shape_b: RectangleShape) -> RectangleShape:
    """Get intersection shape

    Args:
        shape_a (RectangleShape): Rectangle Shape A with coordinates
        shape_b (RectangleShape): Rectangle Shape B with coordinates

    Returns:
        RectangleShape: Intersection area as Rectangle Shape. In case with no intersection, width and height is 0
    """
    x1 = max(shape_a.x, shape_b.x)
    y1 = max(shape_a.y, shape_b.y)
    x2 = min(shape_a.x+shape_a.width, shape_b.x+shape_b.width)
    y2 = min(shape_a.y+shape_a.height, shape_b.y+shape_b.height)
    return RectangleShape(x=x1, y=y1, width=max(0, x2-x1), height=max(0, y2-y1))


def get_related_intersection(shape_a: RectangleShape, shape_b: RectangleShape) -> float:
    """Get related intersection shape_a wrt shape_b. i.e. intersection of figures divided by shape_a

    Args:
        shape_a (RectangleShape): Rectangle Shape A with coordinates (main figure)
        shape_b (RectangleShape): Rectangle Shape B with coordinates (figure for intersection)

    Returns:
        float: ratio
    """
    intersection = get_rectangle_intersection(shape_a, shape_b)
    return float(intersection.width*intersection.height)/(shape_a.width*shape_a.height)


def get_image_border(image_width: int, image_height: int, border_size: int) -> RectangleShape:
    """Get shape related to image with border size

    Args:
        image_width (int): width of image
        image_height (int): height of image
        border_size (int): border size, pixels

    Returns:
        RectangleShape: shape of internal rectangle of frame
    """
    return RectangleShape(border_size, border_size, image_width-border_size*2, image_height-border_size*2)


def intersection_with_border(shape: RectangleShape, image_width: int, image_height: int, border_size: int) -> float:
    """Find max intersection with border

    Args:
        shape (RectangleShape): Shape for intersection
        image_width (int): width of image
        image_height (int): height of image
        border_size (int): border size, pixels

    Returns:
        float: intersection with border
    """
    return 1 - get_related_intersection(shape, get_image_border(image_width, image_height, border_size))


assert get_rectangle_intersection(RectangleShape(0, 0, 8, 8), RectangleShape(
    2, 2, 100, 100)) == RectangleShape(x=2, y=2, width=6, height=6)
assert get_related_intersection(RectangleShape(
    0, 0, 8, 8), RectangleShape(2, 2, 100, 100)) == 6*6/(8.0*8.0)
assert intersection_with_border(
    RectangleShape(1, 0, 8, 8), 1920, 1080, 1) == 0.125


In [8]:
def decode_cv2(image: Image.Image) -> CodeInfo:
    detector = cv2.QRCodeDetector()
    data, bbox, straight_qrcode = detector.detectAndDecode(image_to_cv(image))
    return CodeInfo(type="Unknown CV2 detector", value=data)


def decode_zbar(image: Image.Image) -> CodeInfo:
    decoded_labels = zbar_decode(image)
    codes = []
    for decoded in decoded_labels:
        codes.append(
            CodeInfo(
                type=decoded.type,
                value=decoded.data.decode("utf-8"))
        )
    return codes[0] if len(codes) > 0 else CodeInfo()


def decode_ya(image: Image.Image) -> CodeInfo:
    codes = BarcodeDecoder.decode(crop)
    return CodeInfo() if len(codes) != 1 else codes[0]

In [9]:
CROPS_BASE_DIR = "./crops"
VISUALIZE_BASE_DIR = "./visualisation"

os.makedirs(CROPS_BASE_DIR, exist_ok=True)
os.makedirs(VISUALIZE_BASE_DIR, exist_ok=True)

res_df = pd.DataFrame(columns=result_columns)

for index, row in df.iterrows():
    image = ImageLoader.load(row[a_input])
    image_width, image_height = image.size
    is_empty = row[a_output_empty]
    idx = row[a_id]
    image_filename = os.path.join(
        VISUALIZE_BASE_DIR, f"{idx}_visualisation.jpg")

    if not is_empty:
        toloka_data = json.loads(row[a_output_result])
        shapes = []
        crop_filenames_results = {}
        crop_infos: List[CropInfo] = []

        for label_idx, label in enumerate(toloka_data):
            out = RectangleOutput(label)
            shape = out.to_shape(image_width, image_height)
            crop = image.crop(shape.get_pil_bbox())
            crop_filename = os.path.join(
                CROPS_BASE_DIR, f"{idx}_{label_idx}.jpg")
            crop.save(crop_filename)

            shapes.append(shape)

            code_ya = decode_ya(crop)
            code_cv2 = decode_cv2(crop)
            code_zbar = decode_zbar(crop)

            lap_score = variance_of_laplacian(image_to_cv(crop))
            crop_infos.append(CropInfo(filename=crop_filename,
                              ya_result=code_ya.value,
                              cv2_result=code_cv2.value,
                              zbar_result=code_zbar.value,
                              intersection_with_border=intersection_with_border(
                                  shape, image_width, image_height, BORDER_SIZE),
                              laplacian_score=lap_score))

        # get finale visualisation
        draw = ImageDraw.Draw(image)
        for shape in shapes:
            draw.rectangle(shape.get_pil_bbox(),
                           fill=None,
                           outline=(0, 255, 0),
                           width=3)
        border_shape = get_image_border(image_width, image_height, BORDER_SIZE)
        draw.rectangle(border_shape.get_pil_bbox(),
                       fill=None,
                       outline=(255, 0, 0),
                       width=2)

        crop_info: CropInfo
        for crop_info in crop_infos:
            note = {
                a_id: idx,
                a_input: row[a_input],
                a_res_pack_no: row[a_input].split('/')[-2],
                a_output_empty: is_empty,
                a_output_confidence: row[a_output_confidence],
                a_res_ya_result: crop_info.ya_result,
                a_res_cv_result: crop_info.cv2_result,
                a_res_zbar_result: crop_info.zbar_result,
                a_res_near_border: False if crop_info.intersection_with_border < BORDER_INTERSECTION else True,
                a_res_border_intersection: crop_info.intersection_with_border,
                a_res_crop_filename: crop_info.filename,
                a_res_crop_laplacian_score: crop_info.laplacian_score,
                a_res_visualize_filename: image_filename,
            }
            res_df.loc[len(res_df)] = note
    else:
        note = {
            a_id: idx,
            a_input: row[a_input],
            a_res_pack_no: row[a_input].split('/')[-2],
            a_output_empty: is_empty,
            a_output_confidence: row[a_output_confidence],
            a_res_visualize_filename: image_filename,
        }
        res_df.loc[len(res_df)] = note

    image.save(image_filename)


/var/folders/bn/zxhvyfls7k3dbgj5zp8dz4p5fqzp3g/T/ipykernel_51803/2534549338.py:86: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  res_df.loc[len(res_df)] = note


In [13]:
res_df

,id,image_url,pack_id,output:noObjects,output:confidence,yandex_decoding_result,cv_decoding_result,zbar_decoding_result,near_border,intersection_with_border,crop_filename,laplacian_score,visualisation_filename
0,0,https://mr-toloka-images.s3.yandex.net/QR_PLTD...,pack1,False,0.991624,None,,None,0.0,0.0,./crops/0_0.jpg,34.595949,./visualisation/0_visualisation.jpg
1,1,https://mr-toloka-images.s3.yandex.net/QR_PLTD...,pack1,False,0.992858,None,,None,0.0,0.0,./crops/1_0.jpg,43.069146,./visualisation/1_visualisation.jpg
2,2,https://mr-toloka-images.s3.yandex.net/QR_PLTD...,pack1,False,0.992232,None,,None,0.0,0.0,./crops/2_0.jpg,37.347932,./visualisation/2_visualisation.jpg
3,3,https://mr-toloka-images.s3.yandex.net/QR_PLTD...,pack1,False,0.991751,None,,None,0.0,0.0,./crops/3_0.jpg,36.239353,./visualisation/3_visualisation.jpg
4,4,https://mr-toloka-images.s3.yandex.net/QR_PLTD...,pack1,False,0.991402,None,,None,0.0,0.0,./crops/4_0.jpg,25.857875,./visualisation/4_visualisation.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,257,https://mr-toloka-images.s3.yandex.net/QR_PLTD...,pack5,False,0.993884,None,,None,0.0,0.0,./crops/257_1.jpg,339.880018,./visualisation/257_visualisation.jpg
326,258,https://mr-toloka-images.s3.yandex.net/QR_PLTD...,pack5,False,0.993617,None,,None,0.0,0.0,./crops/258_0.jpg,384.228595,./visualisation/258_visualisation.jpg
327,258,https://mr-toloka-images.s3.yandex.net/QR_PLTD...,pack5,False,0.993617,PLT10398,,None,0.0,0.0,./crops/258_1.jpg,592.856732,./visualisation/258_visualisation.jpg
328,259,https://mr-toloka-images.s3.yandex.net/QR_PLTD...,pack5,False,0.993347,PLT10395,PLT10395,PLT10395,0.0,0.0,./crops/259_0.jpg,376.256968,./visualisation/259_visualisation.jpg


In [23]:
# record results to YMBOT-231 folder
res_df.to_excel("output.xlsx", index=False)


In [10]:
res_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 329
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        330 non-null    int64  
 1   image_url                 330 non-null    object 
 2   pack_id                   330 non-null    object 
 3   output:noObjects          330 non-null    bool   
 4   output:confidence         330 non-null    float64
 5   yandex_decoding_result    66 non-null     object 
 6   cv_decoding_result        328 non-null    object 
 7   zbar_decoding_result      51 non-null     object 
 8   near_border               328 non-null    float64
 9   intersection_with_border  328 non-null    float64
 10  crop_filename             328 non-null    object 
 11  laplacian_score           328 non-null    float64
 12  visualisation_filename    330 non-null    object 
dtypes: bool(1), float64(4), int64(1), object(7)
memory usage: 33.8+ K

In [24]:
quantile_value = 0.75
BORDER_INTERSECTION = 0.4 

packs = res_df[a_res_pack_no].unique()
no_border_filter = res_df[a_res_border_intersection] < BORDER_INTERSECTION
has_ya_result_filter = res_df[a_res_ya_result].str.contains("PLT", na=False)
has_cv_result_filter = res_df[a_res_cv_result].str.contains("PLT", na=False)
has_zbar_result_filter = res_df[a_res_zbar_result].str.contains("PLT", na=False)

filtered_df = None
for pack_name in packs:
    cur_pack_filter = res_df[a_res_pack_no] == pack_name
    laplacian_score_filter_by_quantile = res_df[a_res_crop_laplacian_score] >= res_df.loc[no_border_filter & cur_pack_filter][a_res_crop_laplacian_score].quantile(quantile_value)

    pack_df = res_df.loc[no_border_filter & cur_pack_filter & (has_ya_result_filter | has_cv_result_filter | has_zbar_result_filter | laplacian_score_filter_by_quantile)]
    pack_df.to_excel(f"{pack_name}.xlsx", index=False)

    if filtered_df is None:
        filtered_df = pack_df
    else:
        filtered_df = pd.concat([filtered_df, pack_df])

filtered_df.to_excel("filtered.xlsx", index=False)



In [19]:
filtered_df[a_res_ya_result].count()/len(filtered_df[a_res_ya_result])

0.6632653061224489

In [20]:
filtered_df[a_res_ya_result].count()

65